# Capstone Project with Udacity

# Create a Customer Segmentation Report for Arvato Financial Services

In this project, I will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population of Germany. I'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, I'll apply what I've learned from the segmentation on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to **predict which individuals are most likely to convert into becoming customers for the company**. The data that I'll be using was provided by Bertelsmann Arvato Analytics, and represents a real-life data science task.

In [1]:
# setting random state for reproducibility
random_state = 22

# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

## Part 1: Get to Know the Data

**In this first notebook, I'll be performing the cleaning and pre-processing of the data.**

There are four data files associated with this project:

- `sample_AZDIAS.csv`: Demographics data for the general population of Germany; 222 805 persons (rows) x 366 features (columns).
- `sample_CUSTOMERS.csv`: Demographics data for customers of a mail-order company; 47 913 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. I'll be using the information from the first two files to figure out how customers (`CUSTOMERS`) are similar to or differ from the general population at large (`AZDIAS`), then use the analysis to make predictions on the other two files (`MAILOUT`), predicting which recipients are most likely to become a customer for the mail-order company.

The `CUSTOMERS` file contains three extra columns (`CUSTOMER_GROUP`, `ONLINE_PURCHASE`, and `PRODUCT_GROUP`), which provide broad information about the customers depicted in the file. The original `MAILOUT` file included one additional column, `RESPONSE`, which indicated whether or not each recipient became a customer of the company. For the `TRAIN` subset, this column has been retained, but in the `TEST` subset it has been removed; it is against that withheld column that the final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, there are two Excel spreadsheets provided in the files that can be refered to:

- `DIAS Information Levels - Attributes 2017.xlsx`:  is a top-level list of attributes and descriptions, organized by informational category.
- `DIAS Attributes - Values 2017.xlsx`:  is a detailed mapping of data values for each feature in alphabetical order.

### Load and check rows and columns

In [2]:
# load in the data
azdias = pd.read_csv('data/sample_AZDIAS.csv')
customers = pd.read_csv('data/sample_CUSTOMERS.csv')

del azdias['Unnamed: 0']
del customers['Unnamed: 0']

C:\Users\Qesh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Looks like there's something off about columns 18 and 19. We'll check that out later on.

In [3]:
# check the datasets sizes
print('AZDIAS:     ',azdias.shape)
print('CUSTOMERS:  ',customers.shape)

AZDIAS:      (222805, 366)
CUSTOMERS:   (47913, 369)


In [4]:
azdias.ALTER_HH.isnull().sum()

18538

In [5]:
azdias.ALTER_HH.replace(['None', 'nan'], np.nan, inplace=True)

In [6]:
azdias.ALTER_KIND1

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
222800     NaN
222801     NaN
222802    18.0
222803     NaN
222804     NaN
Name: ALTER_KIND1, Length: 222805, dtype: float64

In [7]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,406859,2,1.0,19.0,NaN,NaN,NaN,NaN,13.0,NaN,...,NaN,2.0,3.0,3.0,2.0,9.0,NaN,3,2,3
1,548618,-1,5.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,...,2.0,6.0,7.0,4.0,6.0,4.0,7.0,3,1,4
2,546543,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,3
3,554058,-1,1.0,21.0,NaN,NaN,NaN,NaN,21.0,2.0,...,1.0,4.0,7.0,4.0,3.0,9.0,3.0,1,1,1
4,426052,-1,1.0,0.0,NaN,NaN,NaN,NaN,25.0,12.0,...,3.0,6.0,9.0,10.0,NaN,5.0,1.0,5,2,1


In [8]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,13824,0,1.0,8.0,NaN,NaN,NaN,NaN,8.0,3.0,...,1.0,6.0,9.0,2.0,3,COSMETIC,SINGLE_BUYER,0,1,4
1,97051,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,2
2,88334,1,1.0,7.0,NaN,NaN,NaN,NaN,6.0,1.0,...,2.0,6.0,9.0,2.0,4,FOOD,SINGLE_BUYER,0,2,4
3,10491,-1,1.0,0.0,NaN,NaN,NaN,NaN,0.0,7.0,...,8.0,6.0,9.0,3.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
4,167860,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,2


In [9]:
# double check on number of unique rows representing unique people
print(f'AZDIAS:\n {azdias.shape[0]} = {azdias.LNR.nunique()}\n {azdias.shape[0] == azdias.LNR.nunique()}')
print(f'\nCUSTOMERS:\n {customers.shape[0]} = {customers.LNR.nunique()}\n {customers.shape[0] == customers.LNR.nunique()}')

AZDIAS:
 222805 = 222805
 True

CUSTOMERS:
 47913 = 47913
 True


In [10]:
# check the columns
print(f'AZDIAS columns:     {azdias.shape[1]}')
print(f'CUSTOMERS columns:  {customers.shape[1]}')

azdias_columns = list(azdias.columns)
customers_columns = list(customers.columns)
extra_customers_columns = np.setdiff1d(customers_columns, azdias_columns, assume_unique=True)

print(f'{customers.shape[1] - azdias.shape[1]} extra columns in CUSTOMERS are: {extra_customers_columns}')

AZDIAS columns:     366
CUSTOMERS columns:  369
3 extra columns in CUSTOMERS are: ['PRODUCT_GROUP' 'CUSTOMER_GROUP' 'ONLINE_PURCHASE']


Now that we've verified that the columns and rows are ok, let's start to check each column to really understand our data.

Let's start by diving a bit on the extra columns of customers dataset:

In [11]:
customers[extra_customers_columns].head()

,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE
0,COSMETIC,SINGLE_BUYER,0
1,COSMETIC_AND_FOOD,MULTI_BUYER,0
2,FOOD,SINGLE_BUYER,0
3,COSMETIC_AND_FOOD,MULTI_BUYER,0
4,COSMETIC_AND_FOOD,MULTI_BUYER,0


In [12]:
# check the summary for these extra columns
customers[extra_customers_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47913 entries, 0 to 47912
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_GROUP    47913 non-null  object
 1   CUSTOMER_GROUP   47913 non-null  object
 2   ONLINE_PURCHASE  47913 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [13]:
customers[extra_customers_columns].PRODUCT_GROUP.unique()

array(['COSMETIC', 'COSMETIC_AND_FOOD', 'FOOD'], dtype=object)

In [14]:
customers[extra_customers_columns].CUSTOMER_GROUP.unique()

array(['SINGLE_BUYER', 'MULTI_BUYER'], dtype=object)

In [15]:
customers[extra_customers_columns].ONLINE_PURCHASE.unique()

array([0, 1], dtype=int64)

As we can see, there's no missing values in these three columns and all of them are categorical columns.

We'll import the excel sheets that contain information about the datasets attributes so we can make better judgments regarding the columns.

In [16]:
data_att = pd.read_excel('data/data_description/DIAS Attributes - Values 2017.xlsx', header=1)
data_info = pd.read_excel('data/data_description/DIAS Information Levels - Attributes 2017.xlsx', header=1)

del data_att['Unnamed: 0']
del data_info['Unnamed: 0']

In [17]:
data_att.head(8)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,0,no classification possible
2,NaN,NaN,1,passive elderly
3,NaN,NaN,2,cultural elderly
4,NaN,NaN,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,1,< 30 years
7,NaN,NaN,2,30 - 45 years


In [18]:
data_info.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


To be able to use these datasets we'll be filling the NaNs with the forward value.

In [19]:
data_att.fillna(method="ffill", inplace=True)
data_att.head(8)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years
7,ALTERSKATEGORIE_GROB,age classification through prename analysis,2,30 - 45 years


In [20]:
data_att.isna().sum()

Attribute      0
Description    0
Value          0
Meaning        0
dtype: int64

In [21]:
data_info.fillna(method="ffill", inplace=True)
data_info.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,Person,ANREDE_KZ,gender,modelled on millions of first name-age-referen...
3,Person,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,Person,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [22]:
data_info[data_info['Information level'].isna()]

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...


In [23]:
data_info.fillna(method="bfill", inplace=True)
data_info.head()

,Information level,Attribute,Description,Additional notes
0,Person,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,Person,ANREDE_KZ,gender,modelled on millions of first name-age-referen...
3,Person,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,Person,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [24]:
data_info.isna().sum()

Information level    0
Attribute            0
Description          0
Additional notes     0
dtype: int64

### Data preparation


The first thing we're going to do is to drop the extra columns from the `customers` dataset so every data wrangling we do, will be done to both datasets (`azdias` - the full population dataset, and `customers` - dataset containing only the already customers).


In [25]:
customers.drop(columns= extra_customers_columns, inplace=True)

In [26]:
customers.shape

(47913, 366)

Let's take a look at the two columns that got a `DtypeWarning` when we loaded the data.

In [27]:
error_columns = list(azdias.iloc[:, 18:20].columns)

for c in error_columns:
    print(f'Column {c}:\n{azdias[c].unique()}\n')

Column CAMEO_DEUG_2015:
[nan 3.0 4.0 8.0 9.0 2.0 5.0 7.0 1.0 6.0 '3.0' '2' '8.0' '1' '4.0' '3'
 '7.0' '9.0' '5' '8' '9' '7' '5.0' '1.0' '4' '6' '2.0' '6.0' 'X']

Column CAMEO_INTL_2015:
[nan 24.0 51.0 55.0 13.0 33.0 54.0 41.0 15.0 14.0 25.0 35.0 43.0 22.0 31.0
 23.0 32.0 44.0 34.0 52.0 12.0 45.0 '25.0' '14' '51.0' '13' '24.0' '25'
 '54.0' '23.0' '24' '32' '54' '41.0' '22.0' '51' '31' '41' '12' '34.0'
 '13.0' '52' '55' '35' '14.0' '45' '43' '23' '55.0' '31.0' '44' '43.0'
 '33' '15.0' '15' '34' '52.0' '45.0' '22' 'XX' '32.0' '44.0' '33.0' '12.0'
 '35.0']



As we can see, there are strange values (X and XX) for both these columns. Let's check the `data_att` to understand what kind of entries are expected for these columns.

In [28]:
data_att[data_att.Attribute == 'CAMEO_DEUG_2015']

,Attribute,Description,Value,Meaning
51,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
52,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,1,upper class
53,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,2,upper middleclass
54,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,3,established middleclasse
55,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,4,consumption-oriented middleclass
56,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,5,active middleclass
57,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,6,low-consumption middleclass
58,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,7,lower middleclass
59,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,8,working class
60,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,9,urban working class


In [29]:
data_att[data_att.Attribute == 'CAMEO_INTL_2015']

,Attribute,Description,Value,Meaning


In [30]:
data_info[data_info.Attribute == 'CAMEO_INTL_2015']

,Information level,Attribute,Description,Additional notes


Something is wrong with this second column. It looks like we can't find it in neither of the descriptions datasets.

Let's see if we can figure it out.

In [31]:
data_info[data_info.Attribute.str.startswith('CAMEO_')]

,Information level,Attribute,Description,Additional notes
85,Microcell (RR4_ID),CAMEO_DEUG_2015,CAMEO_4.0: uppergroup,New German CAMEO Typology established together...
86,Microcell (RR4_ID),CAMEO_DEU_2015,CAMEO_4.0: specific group,New German CAMEO Typology established together...
87,Microcell (RR4_ID),CAMEO_DEUINTL_2015,CAMEO_4.0: international classification,New German CAMEO Typology established together...


It looks like `CAMEO_INTL_2015` is named `CAMEO_DEUINTL_2015` on the `data_att` and `data_info` datasets. Let's try again.

In [32]:
data_att[data_att.Attribute == 'CAMEO_DEUINTL_2015']

,Attribute,Description,Value,Meaning
105,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
106,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
107,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,12,Wealthy Households-Young Couples With Children
108,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,13,Wealthy Households-Families With School Age Ch...
109,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,14,Wealthy Households-Older Families & Mature Co...
110,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,15,Wealthy Households-Elders In Retirement
111,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,21,Prosperous Households-Pre-Family Couples & Sin...
112,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,22,Prosperous Households-Young Couples With Children
113,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,23,Prosperous Households-Families With School Age...
114,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,24,Prosperous Households-Older Families & Mature ...


Alright, that worked!

So we can see that both of them are categorical columns and they should be integers.

Let's then replace the `X` and `XX` values we found with `-1` that is the correct value for `unknown`.

In [33]:
def replace_wrong_values(df, columns_to_fix, wrong_values, correct_value):
    """
    Method to fix wrong entry values of dataframes' columns.
    
    Args:
        df: dataframe to have columns corrected
        columns_to_fix (list): list of columns names that will be fixed
        wrong_values (list): list of values to be replaced
        correct_value: value to replace the wrong ones

    Returns:
        df: fixed dataframe 
    """
    for wv in wrong_values:
        df[columns_to_fix] = df[columns_to_fix].replace(wv, correct_value)
    
    df[columns_to_fix] = df[columns_to_fix].astype(float)
        
    return df

In [34]:
azdias = replace_wrong_values(azdias, error_columns, ['X', 'XX'], -1)
customers = replace_wrong_values(customers, error_columns, ['X', 'XX'], -1)

Now it would be helpful if we took the columns with NaNs and replaced them with `-1` that is the correct value for `unknown`.

For that, we'll search for columns that have NaNs and columns that have the `unknown` attribute meaning and get the intersection between them, wich means, get only the columns that satisfy both of these conditions and then go ahead and replace them.

In [35]:
has_unknown_att = list(data_att[data_att.Meaning == 'unknown'].Attribute)
print(len(has_unknown_att), 'columns have the unknown attribute meaning')

232 columns have the unknown attribute meaning


In [36]:
na_columns = list(azdias.columns[(azdias.isna().sum()>0)])
print(len(na_columns), 'columns have NaN values')

273 columns have NaN values


In [37]:
na_and_unknown = np.intersect1d(has_unknown_att, na_columns)
print(len(na_and_unknown), 'columns have the unknown attribute meaning AND NaN values\nThese are the columns we can replace the NaNs with -1 (that mean unknown)')

194 columns have the unknown attribute meaning AND NaN values
These are the columns we can replace the NaNs with -1 (that mean unknown)


In [38]:
na_not_unknown = np.setdiff1d(na_columns, has_unknown_att)
print(len(na_not_unknown), "columns have NaN values but don't have the unknown attribute meaning.\nWe'll need to dig a little deeper to figure out what todo in this cases.")

79 columns have NaN values but don't have the unknown attribute meaning.
We'll need to dig a little deeper to figure out what todo in this cases.


In [39]:
def replace_NaNs_with_unknown(df, data_att):
    """
    Method to replace NaNs with -1 (meaning that the value is unknown).
    
    Args:
        df: dataframe to have NaNs replaced
        data_att: dataframe with attributes description

    Returns:
        df: replaced dataframe 
    """
 
    # make a list of columns for both situations
    has_unknown_att = list(data_att[data_att.Meaning == 'unknown'].Attribute)
    na_columns = list(df.columns[(df.isna().sum()>0)])
    
    # get the intersection list of columns
    na_and_unknown = np.intersect1d(has_unknown_att, na_columns)
    
    # replace all NaNs with -1
    df[na_and_unknown] = df[na_and_unknown].fillna(-1)
        
    return df

In [40]:
azdias = replace_NaNs_with_unknown(azdias, data_att)
customers = replace_NaNs_with_unknown(customers, data_att)

#### Missing data

Now that we have dealt with missing data on columns where `unknown` was an attribute option (in that case, we're considering that having an unknown value means something because it was given as an option to the categorical columns), we need to look deeper into columns with actual missing values.

In [41]:
azdias.columns[(azdias.isnull().sum()/azdias.shape[0])>0.4]

Index(['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992',
       'KK_KUNDENTYP'],
      dtype='object')

In [42]:
customers.columns[(customers.isnull().sum()/customers.shape[0])>0.4]

Index(['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992',
       'KK_KUNDENTYP'],
      dtype='object')

In [43]:
# make lists of columns that are above the percentage threshold of 40%
azdias_null_cols = list(azdias.columns[(azdias.isnull().sum()/azdias.shape[0])>0.4])
customers_null_cols = list(customers.columns[(customers.isnull().sum()/customers.shape[0])>0.4])

# get the intersection list of columns
common_null_cols = list(np.intersect1d(azdias_null_cols, customers_null_cols))

Now that we've selected columns with over 40% of missing values to drop, there's still a lot of columns we will need to inspect.

For this task, let's start taking a look at what these columns mean so we can understand how to best deal with them.

In [44]:
print('Number of cols with missing values:', len(azdias.columns[(azdias.isnull().sum()/azdias.shape[0])>0]))

Number of cols with missing values: 79


In [45]:
print('Number of rows with over 20% of missing values:', len(azdias.index[(azdias.isnull().sum(axis=1)/azdias.shape[1])>0.20]))
print('Number of rows with over 18% of missing values:', len(azdias.index[(azdias.isnull().sum(axis=1)/azdias.shape[1])>0.18]))
print('Number of rows with over 15% of missing values:', len(azdias.index[(azdias.isnull().sum(axis=1)/azdias.shape[1])>0.15]))

Number of rows with over 20% of missing values: 0
Number of rows with over 18% of missing values: 3
Number of rows with over 15% of missing values: 18549


Looking at missing values in rows, there's not a significant amount of rows with over 18% of missing values.

When we lower that percentage a bit, we can see that the amount of rows start to to up. Rows that have over 15% of missing values are way more significant.

In this situation it's better to handle missing values column wise instead of row wise, so we don't end up losing entry data. When we handle the columns missing values, it will fix the rows missing values as well.

In [46]:
print('Null values on azdias dataframe:', azdias.isnull().sum().sum())
print('Percentage of null values on azdias dataframe:', azdias.shape[0]*azdias.shape[1] / azdias.isnull().sum().sum())

Null values on azdias dataframe: 2911777
Percentage of null values on azdias dataframe: 28.00579508664297


In [47]:
((azdias.isnull().sum()/azdias.shape[0])).sort_values(ascending=False).head(20)

ALTER_KIND4                    0.998730
ALTER_KIND3                    0.993353
ALTER_KIND2                    0.967052
ALTER_KIND1                    0.909104
EXTSEL992                      0.734382
KK_KUNDENTYP                   0.658096
ALTERSKATEGORIE_FEIN           0.296165
D19_SOZIALES                   0.289406
D19_TELKO_ONLINE_QUOTE_12      0.289406
D19_GESAMT_ONLINE_QUOTE_12     0.289406
D19_KONSUMTYP                  0.289406
D19_LETZTER_KAUF_BRANCHE       0.289406
D19_LOTTO                      0.289406
D19_BANKEN_ONLINE_QUOTE_12     0.289406
D19_VERSAND_ONLINE_QUOTE_12    0.289406
D19_VERSI_ONLINE_QUOTE_12      0.289406
MOBI_REGIO                     0.150445
VHN                            0.136824
PLZ8_BAUMAX                    0.131204
HH_DELTA_FLAG                  0.121146
dtype: float64

After some digging into each column, we came up with the columns to make different inputations.

Some of the columns we'll simply imput the value for `unknown` as we could see in the `data_att` dataframe.

Other columns we could identify a pattern between it and a similar column, so we made a dictionary with the pair of columns where the key column is the one we want to input and the value column is the one we're basing the inputation from.

At the end, we'll take the remaining columns with missing values and input the most frequent value. Because they're categorical columns, it makes sense to input the most frequent and not the mean or the median, for exemple, as that would input a non existing value that would make no sense as a category.

In [48]:
# columns to make inputations
cols_to_drop = common_null_cols
cols_to_drop.append('D19_SOZIALES')

cols_to_input_zero = ['ALTERSKATEGORIE_FEIN',
                      'D19_LOTTO',
                      'HH_DELTA_FLAG',
                      'D19_TELKO_ONLINE_QUOTE_12',
                      'D19_GESAMT_ONLINE_QUOTE_12',
                      'D19_BANKEN_ONLINE_QUOTE_12',
                      'D19_VERSAND_ONLINE_QUOTE_12',
                      'D19_VERSI_ONLINE_QUOTE_12']

In [49]:
def handle_missing_data_cols(df, cols_to_drop, cols_to_input_zero):
    """
    Method to handle columns that have missing data.
    
    Args:
        df: dataframe to have missing data handled
        cols_to_drop (list): list of columns to be dropped from dataframe
        columns_to_input_zero (list): list of columns to input zero (unknown)

    Returns:
        df: dataframe with missing data handled
    """
    
    # drop columns from df
    print(f'Dropping {len(cols_to_drop)} columns for having over 40% of null values')
    df = df.drop(columns=cols_to_drop)
    
        
    # input fixed value to columns
    values = {
        'D19_KONSUMTYP': 9,
        'D19_LETZTER_KAUF_BRANCHE': 'D19_UNBEKANNT',
        'MOBI_REGIO': 6
    }
    
    # adding columns to input zero to the dict
    for col in cols_to_input_zero:
        values[col] = 0
    
    print(f'Imputting fixed values to {len(values.keys())} columns.\n')
    df.fillna(value=values, inplace=True)
    
    return df

In [50]:
azdias = handle_missing_data_cols(azdias, cols_to_drop, cols_to_input_zero)
customers = handle_missing_data_cols(customers, cols_to_drop, cols_to_input_zero)

Dropping 7 columns for having over 40% of null values
Imputting fixed values to 11 columns.

Dropping 7 columns for having over 40% of null values
Imputting fixed values to 11 columns.



In [51]:
def make_imputations(df):
    """
    Method to imput most frequent value to the remaining columns with missing data.
    
    Args:
        df: dataframe to have missing data handled

    Returns:
        df: dataframe with missing data handled
    """
    
    # make sure all NaNs are actually NaNs
    df.replace(['None', 'nan'], np.nan, inplace=True)
    
    # transform columns dtype object to string
    df['OST_WEST_KZ'] = df['OST_WEST_KZ'].astype('str')
    
    # input most common value to remaining missing columns
    remaining_null_cols = list(df.columns[(df.isnull().sum())>0])
    print(f'Imputting most frequent values to remaining {len(remaining_null_cols)} columns\nThis might take a while...\n')
    
    imputer = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
    df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns, index = df.index)
    
    return df

In [52]:
azdias = make_imputations(azdias)
customers = make_imputations(customers)

Imputting most frequent values to remaining 61 columns
This might take a while...

Imputting most frequent values to remaining 61 columns
This might take a while...



Now let's check how our missing values are.

In [53]:
print('Null values on azdias dataframe:    ', azdias.isnull().sum().sum())
print('Null values on customers dataframe: ', azdias.isnull().sum().sum())

Null values on azdias dataframe:     0
Null values on customers dataframe:  0


We're all done dealing with missing data now!

#### Feature Selection

It's time to take a look at the columns available and to choose witch ones to keep for our purpose here.

In [54]:
unwanted_cols = [
    'ALTER_HH', 'ALTERSKATEGORIE_FEIN', 'ANZ_STATISTISCHE_HAUSHALTE', 'CAMEO_DEU_2015',
    'CAMEO_INTL_2015', 'D19_LETZTER_KAUF_BRANCHE', 'EINGEFUEGT_AM', 'LP_LEBENSPHASE_FEIN']

# anything above 10 will be considered 10
cols_change_max_10 = ['ANZ_HH_TITEL']

# anything above 10 will be considered 5
cols_change_max_5 = ['ANZ_PERSONEN']

# group column according to data_att
# 1-2, 3-5, 6-7, 8-9, 10
cols_to_group = ['LP_STATUS_GROB']

After digging into the columns, there are some final changes we need to make before we prepare our datasets for the task.

We've selected a couple more columns to drop due to redundance or because it doesn't show potential to the task. Others were selected to be cleaned up a bit more in terms of entry values.

In [55]:
def more_data_cleaning(df):
    """
    Method to handle final data cleaning columns.
    
    Args:
        df: dataframe to be cleaned

    Returns:
        df: cleaned dataframe
    """
    
    # drop unwanted columns from df
    print(f'Dropping {len(unwanted_cols)} columns.')
    df = df.drop(columns=unwanted_cols)
    
    # changing values to match a maximum of 10
    print(f'Changing values to 10 on {len(cols_change_max_10)} column.')
    df[cols_change_max_10] = np.where(df[cols_change_max_10]>=10.0 , 10.0, df[cols_change_max_10])
    
    # changing values to match a maximum of 5
    print(f'Changing values to 5 on {len(cols_change_max_5)} column.')
    df[cols_change_max_5] = np.where(df[cols_change_max_5]>=5.0 , 5.0, df[cols_change_max_5])
    
    # changing values of column according to data_att
    print(f'Changing values to group common entries on {len(cols_to_group)} column.')
    df[cols_to_group] = np.where(df[cols_to_group]==2.0 , 1.0, df[cols_to_group])
    df[cols_to_group] = np.where(df[cols_to_group]==4.0 , 3.0, df[cols_to_group])
    df[cols_to_group] = np.where(df[cols_to_group]==5.0 , 3.0, df[cols_to_group])
    df[cols_to_group] = np.where(df[cols_to_group]==7.0 , 6.0, df[cols_to_group])
    df[cols_to_group] = np.where(df[cols_to_group]==9.0 , 8.0, df[cols_to_group])
    
    return df

In [56]:
azdias = more_data_cleaning(azdias)
customers = more_data_cleaning(customers)

Dropping 8 columns.
Changing values to 10 on 1 column.
Changing values to 5 on 1 column.
Changing values to group common entries on 1 column.
Dropping 8 columns.
Changing values to 10 on 1 column.
Changing values to 5 on 1 column.
Changing values to group common entries on 1 column.


In [57]:
azdias.shape

(222805, 351)

In [58]:
customers.shape

(47913, 351)

#### Categorical Features

Let's handle some of the categorical features that are in innapropriate formats.

In [59]:
# check for columns with less than 3 categories

binary_cat_cols = []

for col in azdias.columns:
    if azdias[col].nunique() <= 3:
        binary_cat_cols.append(col)
        print(col)
        
print(binary_cat_cols)

DSL_FLAG
GREEN_AVANTGARDE
HH_DELTA_FLAG
KBA13_ANTG4
KONSUMZELLE
LP_STATUS_GROB
OST_WEST_KZ
SOHO_KZ
STRUKTURTYP
UNGLEICHENN_FLAG
VERS_TYP
ANREDE_KZ
['DSL_FLAG', 'GREEN_AVANTGARDE', 'HH_DELTA_FLAG', 'KBA13_ANTG4', 'KONSUMZELLE', 'LP_STATUS_GROB', 'OST_WEST_KZ', 'SOHO_KZ', 'STRUKTURTYP', 'UNGLEICHENN_FLAG', 'VERS_TYP', 'ANREDE_KZ']


In [60]:
for col in binary_cat_cols:
    print(azdias[col].value_counts())
    print('\n')

1.0    216372
0.0      6433
Name: DSL_FLAG, dtype: int64


0    178809
1     43996
Name: GREEN_AVANTGARDE, dtype: int64


0.0    204652
1.0     18153
Name: HH_DELTA_FLAG, dtype: int64


0.0    121899
1.0     69411
2.0     31495
Name: KBA13_ANTG4, dtype: int64


0.0    175977
1.0     46828
Name: KONSUMZELLE, dtype: int64


1.0    142241
3.0     80564
Name: LP_STATUS_GROB, dtype: int64


W     157385
O      41997
-1     23423
Name: OST_WEST_KZ, dtype: int64


0.0    221109
1.0      1696
Name: SOHO_KZ, dtype: int64


3.0    163145
1.0     32002
2.0     27658
Name: STRUKTURTYP, dtype: int64


0.0    204572
1.0     18233
Name: UNGLEICHENN_FLAG, dtype: int64


 2    99437
 1    95336
-1    28032
Name: VERS_TYP, dtype: int64


2    116435
1    106370
Name: ANREDE_KZ, dtype: int64




In [61]:
data_att[data_att.Attribute.str.contains('OST_WEST_KZ')]

,Attribute,Description,Value,Meaning
2013,OST_WEST_KZ,flag indicating the former GDR/FRG,-1,unknown
2014,OST_WEST_KZ,flag indicating the former GDR/FRG,O,East (GDR)
2015,OST_WEST_KZ,flag indicating the former GDR/FRG,W,West (FRG)


In [62]:
data_att[data_att.Attribute.str.contains('VERS_TYP')]

,Attribute,Description,Value,Meaning
2216,VERS_TYP,insurance typology,-1,unknown
2217,VERS_TYP,insurance typology,1,social-safety driven
2218,VERS_TYP,insurance typology,2,individualistic-accepting risks


In [63]:
data_att[data_att.Attribute.str.contains('ANREDE_KZ')]

,Attribute,Description,Value,Meaning
33,ANREDE_KZ,gender,"-1, 0",unknown
34,ANREDE_KZ,gender,1,male
35,ANREDE_KZ,gender,2,female


Here's what we'll be doing to these three columns:

`OST_WEST_KZ`: will turn `O` and `W` into `0` and `1`.

`VERS_TYP` and `ANREDE_KZ`: will turn `2` into `0`.

In [64]:
def fix_binary_cols(df):
    """
    Method to fix binary category columns.
    
    Args:
        df: dataframe to be fixed

    Returns:
        df: fixed dataframe
    """
    
    df['OST_WEST_KZ'] = df['OST_WEST_KZ'].replace({'W': 1,
                                                   'O': 0})
    
    df['VERS_TYP'] = df['VERS_TYP'].replace(2, 0)
    
    df['ANREDE_KZ'] = df['ANREDE_KZ'].replace(2, 0)
    
    return df

In [65]:
azdias = fix_binary_cols(azdias)
customers = fix_binary_cols(customers)

In [66]:
# check for columns with over 10 categories

many_cat_cols = []

for col in azdias.columns:
    if azdias[col].nunique() > 11:
        many_cat_cols.append(col)
        print(col)
        
print(many_cat_cols)

LNR
ANZ_HAUSHALTE_AKTIV
EINGEZOGENAM_HH_JAHR
GEBURTSJAHR
GFK_URLAUBERTYP
KBA13_ANZAHL_PKW
LP_FAMILIE_FEIN
LP_LEBENSPHASE_GROB
MIN_GEBAEUDEJAHR
PRAEGENDE_JUGENDJAHRE
VERDICHTUNGSRAUM
VK_DISTANZ
['LNR', 'ANZ_HAUSHALTE_AKTIV', 'EINGEZOGENAM_HH_JAHR', 'GEBURTSJAHR', 'GFK_URLAUBERTYP', 'KBA13_ANZAHL_PKW', 'LP_FAMILIE_FEIN', 'LP_LEBENSPHASE_GROB', 'MIN_GEBAEUDEJAHR', 'PRAEGENDE_JUGENDJAHRE', 'VERDICHTUNGSRAUM', 'VK_DISTANZ']


In [67]:
data_att[data_att.Attribute.str.contains('PRAEGENDE_JUGENDJAHRE')]

,Attribute,Description,Value,Meaning
2054,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,"-1, 0",unknown
2055,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,1,"40ies - war years (Mainstream, O+W)"
2056,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,2,"40ies - reconstruction years (Avantgarde, O+W)"
2057,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,3,"50ies - economic miracle (Mainstream, O+W)"
2058,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,4,50ies - milk bar / Individualisation (Avantgar...
2059,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,5,"60ies - economic miracle (Mainstream, O+W)"
2060,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,6,60ies - generation 68 / student protestors (Av...
2061,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,7,60ies - opponents to the building of the Wall ...
2062,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,8,"70ies - family orientation (Mainstream, O+W)"
2063,PRAEGENDE_JUGENDJAHRE,dominating movement in the person's youth (ava...,9,"70ies - peace movement (Avantgarde, O+W)"


The column above has too many categories so we'll split it into two columns: one regarding the movement (avantgard or mainstream) and the decade of infuence.

In [68]:
decade = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 3,
    6: 3,
    7: 3,
    8: 4,
    9: 4,
    10: 5,
    11: 5,
    12: 5,
    13: 5,
    14: 6,
    15: 6
}

mainstream = {
    1: 1,
    2: 0,
    3: 1,
    4: 0,
    5: 1,
    6: 0,
    7: 0,
    8: 1,
    9: 0,
    10: 1,
    11: 0,
    12: 1,
    13: 0,
    14: 1,
    15: 0
}

In [69]:
def create_decade_movement(df):
    """
    Method to create two columns out of a columns with too many categories.
    
    Args:
        df: dataframe to be fixed

    Returns:
        df: fixed dataframe
    """
    
    df['PRAEGENDE_JUGENDJAHRE_DECADE'] = df['PRAEGENDE_JUGENDJAHRE'].replace(decade)
    df['PRAEGENDE_JUGENDJAHRE_MAINSTREAM'] = df['PRAEGENDE_JUGENDJAHRE'].replace(mainstream)
    df.drop(columns='PRAEGENDE_JUGENDJAHRE', inplace=True)
    
    return df

In [70]:
azdias = create_decade_movement(azdias)
customers = create_decade_movement(customers)

#### Columns types

It's important that we make sure our columns are numerical so that the unsupervised learning models for clustering works.

In [71]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222805 entries, 0 to 222804
Columns: 352 entries, LNR to PRAEGENDE_JUGENDJAHRE_MAINSTREAM
dtypes: int64(4), object(348)
memory usage: 598.4+ MB


In [72]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47913 entries, 0 to 47912
Columns: 352 entries, LNR to PRAEGENDE_JUGENDJAHRE_MAINSTREAM
dtypes: int64(4), object(348)
memory usage: 128.7+ MB


Most of our columns are object dtype. As we've alredy made all necessary changes previously we can go ahead and change the columns to numeric dtype.

In [73]:
def change_cols_dtypes(df):
    """
    Method to change columns dtypes to numeric.
    
    Args:
        df: dataframe to be fixed

    Returns:
        df: fixed dataframe
    """
    
    df = df.apply(pd.to_numeric)
    
    return df

In [74]:
azdias = change_cols_dtypes(azdias)
customers = change_cols_dtypes(customers)

In [75]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222805 entries, 0 to 222804
Columns: 352 entries, LNR to PRAEGENDE_JUGENDJAHRE_MAINSTREAM
dtypes: float64(257), int64(95)
memory usage: 598.4 MB


In [76]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47913 entries, 0 to 47912
Columns: 352 entries, LNR to PRAEGENDE_JUGENDJAHRE_MAINSTREAM
dtypes: float64(257), int64(95)
memory usage: 128.7 MB


#### More feature selection

We're gonna check out some columns that are highly correlated between them. We should avoid having those because it can interfere on the clustering due to redundancy.

In [77]:
def get_high_corr_cols(df, t):
    """
    Method to get the columns with chigh correlation coeficient (above the threshold).
    
    Args:
        df: dataframe to be fixed
        t: threshold for correlation coeficient

    Returns:
        df: fixed dataframe
    """

    # select the upper triangle from the correlation matrix
    upper = df.corr(method='spearman').abs().where(np.triu(np.ones(azdias.corr(method='spearman').abs().shape), k=1).astype(bool))

    # find columns with correlation coeficient higher than threshold
    pos_high_corr_cols = [column for column in upper.columns if any(upper[column] > t)]
    neg_high_corr_cols = [column for column in upper.columns if any(upper[column] < -t)]
    
    high_corr_cols = pos_high_corr_cols + neg_high_corr_cols
    
    return high_corr_cols

In [78]:
high_corr_cols = get_high_corr_cols(azdias, 0.85)

In [79]:
azdias[high_corr_cols]

,CJT_TYP_2,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_ONLINE_DATUM,D19_KONSUMTYP,D19_KONSUMTYP_MAX,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,...,KBA13_SEG_KLEINWAGEN,KBA13_SEG_VAN,KBA13_VW,LP_FAMILIE_FEIN,LP_FAMILIE_GROB,LP_LEBENSPHASE_GROB,ORTSGR_KLS9,PLZ8_BAUMAX,PLZ8_HHZ,PRAEGENDE_JUGENDJAHRE_DECADE
0,2.0,6,0.0,4,4,6,2.0,3,1,3,...,-1.0,-1.0,-1.0,11.0,5.0,12.0,-1.0,1.0,-1.0,4
1,1.0,10,0.0,2,5,8,3.0,2,2,2,...,3.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,5.0,3
2,5.0,10,0.0,0,10,10,9.0,9,0,0,...,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0
3,4.0,10,0.0,2,5,5,1.0,4,2,2,...,4.0,3.0,3.0,5.0,3.0,6.0,5.0,1.0,3.0,6
4,5.0,7,0.0,1,7,7,5.0,3,0,0,...,2.0,2.0,3.0,1.0,1.0,1.0,7.0,2.0,4.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222800,3.0,10,0.0,0,10,10,9.0,9,0,0,...,-1.0,-1.0,-1.0,1.0,1.0,2.0,-1.0,1.0,-1.0,0
222801,4.0,10,0.0,0,9,10,9.0,8,0,0,...,3.0,3.0,4.0,5.0,3.0,6.0,4.0,1.0,5.0,6
222802,5.0,10,0.0,1,2,2,3.0,2,1,1,...,2.0,3.0,2.0,5.0,3.0,6.0,9.0,3.0,3.0,6
222803,2.0,10,0.0,0,8,10,9.0,8,0,0,...,3.0,4.0,3.0,10.0,5.0,12.0,1.0,1.0,1.0,0


In [80]:
def drop_high_corr_cols(df, high_corr_cols):
    """
    Method to drop highly correlated columns.
    
    Args:
        df: dataframe to be fixed

    Returns:
        df: fixed dataframe
    """

    df.drop(columns=high_corr_cols, inplace=True)
    
    return df

In [81]:
azdias = drop_high_corr_cols(azdias, high_corr_cols)
customers = drop_high_corr_cols(customers, high_corr_cols)

And finally, we'll drop `LNR` column as it's a sort of id column, so it won't do us any good for now.

In [82]:
def drop_id_col(df):
    """
    Method to drop column LNR that is an id column.
    
    Args:
        df: dataframe to be fixed

    Returns:
        df: fixed dataframe
    """
    df.drop(columns='LNR', inplace=True)
    
    return df

In [83]:
azdias = drop_id_col(azdias)
customers = drop_id_col(customers)

Now let's go ahead and apply all those steps to both the TRAIN and TEST files provided as we'll need them for the last part of the project.

In [84]:
# load in the data
df_train = pd.read_csv('data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';', low_memory=False)
df_test = pd.read_csv('data/Udacity_MAILOUT_052018_TEST.csv', sep=';', low_memory=False)

In [85]:
# check the datasets sizes
print('TRAIN: ', df_train.shape)
print('TEST:  ', df_test.shape)

TRAIN:  (42962, 367)
TEST:   (42833, 366)


In [86]:
df_train.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1763,2,1.0,8.0,NaN,NaN,NaN,NaN,8.0,15.0,...,5.0,2.0,1.0,6.0,9.0,3.0,3,0,2,4
1,1771,1,4.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,1.0,4.0,9.0,7.0,1,0,2,3
2,1776,1,1.0,9.0,NaN,NaN,NaN,NaN,7.0,0.0,...,6.0,4.0,2.0,NaN,9.0,2.0,3,0,1,4
3,1460,2,1.0,6.0,NaN,NaN,NaN,NaN,6.0,4.0,...,8.0,11.0,11.0,6.0,9.0,1.0,3,0,2,4
4,1783,2,1.0,9.0,NaN,NaN,NaN,NaN,9.0,53.0,...,2.0,2.0,1.0,6.0,9.0,3.0,3,0,1,3


In [87]:
df_test.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1754,2,1.0,7.0,NaN,NaN,NaN,NaN,6.0,2.0,...,4.0,5.0,6.0,3.0,6.0,9.0,3.0,3,1,4
1,1770,-1,1.0,0.0,NaN,NaN,NaN,NaN,0.0,20.0,...,1.0,5.0,2.0,1.0,6.0,9.0,5.0,3,1,4
2,1465,2,9.0,16.0,NaN,NaN,NaN,NaN,11.0,2.0,...,3.0,9.0,6.0,3.0,2.0,9.0,4.0,3,2,4
3,1470,-1,7.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,...,2.0,6.0,6.0,3.0,NaN,9.0,2.0,3,2,4
4,1478,1,1.0,21.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,4.0,3.0,3.0,9.0,7.0,4,2,4


In [88]:
# applying all cleaning functions to train and test datasets
df_train = replace_wrong_values(df_train, error_columns, ['X', 'XX'], -1)
df_test = replace_wrong_values(df_test, error_columns, ['X', 'XX'], -1)

In [89]:
df_train = replace_NaNs_with_unknown(df_train, data_att)
df_test = replace_NaNs_with_unknown(df_test, data_att)

In [90]:
df_train = handle_missing_data_cols(df_train, cols_to_drop, cols_to_input_zero)
df_test = handle_missing_data_cols(df_test, cols_to_drop, cols_to_input_zero)

Dropping 7 columns for having over 40% of null values
Imputting fixed values to 11 columns.

Dropping 7 columns for having over 40% of null values
Imputting fixed values to 11 columns.



In [91]:
df_train = make_imputations(df_train)
df_test = make_imputations(df_test)

Imputting most frequent values to remaining 61 columns
This might take a while...

Imputting most frequent values to remaining 61 columns
This might take a while...



In [92]:
df_train = more_data_cleaning(df_train)
df_test = more_data_cleaning(df_test)

Dropping 8 columns.
Changing values to 10 on 1 column.
Changing values to 5 on 1 column.
Changing values to group common entries on 1 column.
Dropping 8 columns.
Changing values to 10 on 1 column.
Changing values to 5 on 1 column.
Changing values to group common entries on 1 column.


In [93]:
df_train = fix_binary_cols(df_train)
df_test = fix_binary_cols(df_test)

In [94]:
df_train = create_decade_movement(df_train)
df_test = create_decade_movement(df_test)

In [95]:
df_train = change_cols_dtypes(df_train)
df_test = change_cols_dtypes(df_test)

In [97]:
df_train = drop_high_corr_cols(df_train, high_corr_cols)
df_test = drop_high_corr_cols(df_test, high_corr_cols)

In [98]:
df_train = drop_id_col(df_train)
df_test = drop_id_col(df_test)

In [103]:
# final datasets
print('Final AZDIAS:    ',azdias.shape)
print('Final CUSTOMERS: ',customers.shape)
print('Final TRAIN:     ',df_train.shape)
print('Final TEST:      ',df_test.shape)

azdias.to_csv('data/clean_AZDIAS.csv')
customers.to_csv('data/clean_CUSTOMERS.csv')
df_train.to_csv('data/clean_TRAIN.csv')
df_train.to_csv('data/clean_TEST.csv')

print('\n----> All datasets were exported successfully!')

Final AZDIAS:     (222805, 309)
Final CUSTOMERS:  (47913, 309)
Final TRAIN:      (42962, 310)
Final TEST:       (42833, 309)

----> All datasets were exported successfully!
